In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [2]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name)

In [3]:
startDate = '2023-12-20'
endDate = '2023-12-22'

In [4]:
lookahead_sked = homeOnlySked.loc[(homeOnlySked['gameDate'] >= startDate) & (homeOnlySked['gameDate'] <= endDate)]
lookahead_sked.sort_values('gameTime')

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
632,2023020510,2023-12-22,Friday 07:00 PM,PHI,DET,2023-12-23 00:00:00,0
95,2023020511,2023-12-22,Friday 07:30 PM,EDM,NYR,2023-12-23 00:30:00,0
263,2023020512,2023-12-22,Friday 08:00 PM,BOS,WPG,2023-12-23 01:00:00,0
1286,2023020513,2023-12-22,Friday 08:30 PM,MTL,CHI,2023-12-23 01:30:00,0
1125,2023020499,2023-12-21,Thursday 07:00 PM,WSH,CBJ,2023-12-22 00:00:00,0
958,2023020498,2023-12-21,Thursday 07:00 PM,TOR,BUF,2023-12-22 00:00:00,0
382,2023020500,2023-12-21,Thursday 07:00 PM,STL,FLA,2023-12-22 00:00:00,0
425,2023020502,2023-12-21,Thursday 07:00 PM,NSH,PHI,2023-12-22 00:00:00,0
875,2023020503,2023-12-21,Thursday 07:00 PM,CAR,PIT,2023-12-22 00:00:00,0
712,2023020504,2023-12-21,Thursday 07:00 PM,VGK,TBL,2023-12-22 00:00:00,0


In [5]:
teams_data = {}

for index, row in lookahead_sked.iterrows():

    home_team = row['homeTeam']
    away_team = row['awayTeam']

    if home_team not in teams_data:
        teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
    if away_team not in teams_data:
        teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

    teams_data[home_team]['played'] += 1
    teams_data[away_team]['played'] += 1
    teams_data[home_team]['home'] += 1
    teams_data[away_team]['away'] += 1

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

for index, row in teams_df.iterrows():
    abbrev = index
    full_name = team_names.loc[team_names['abbreviation'] == abbrev]['name'].iloc[0]
    teams_df.at[index, 'team_name'] = full_name
    
teams_df

,played,home,away,team_name
WPG,2,2,0,Winnipeg Jets
DET,2,1,1,Detroit Red Wings
PHI,2,1,1,Philadelphia Flyers
WSH,2,1,1,Washington Capitals
EDM,2,0,2,Edmonton Oilers
MTL,2,0,2,Montréal Canadiens
NYR,1,1,0,New York Rangers
DAL,1,1,0,Dallas Stars
COL,1,1,0,Colorado Avalanche
LAK,1,1,0,Los Angeles Kings


In [6]:
team_check = teams_df['team_name'].to_list()

nhl_teams = [
    'New Jersey Devils', 'New York Islanders', 'New York Rangers',
    'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins',
    'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators',
    'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers',
    'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks',
    'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues',
    'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
    'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars',
    'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets',
    'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes',
    'Vegas Golden Knights', 'Seattle Kraken'
]

# # teams_df.index.values

for team in nhl_teams:

    if team not in team_check:
        print(team)

In [7]:
all_df_even = all_df.copy()
all_df_even['evenGoals'] = all_df_even['goals'] - all_df_even['powerPlayGoals'] - all_df_even['shorthandedGoals']
all_df_even['powerPlayAssists'] = all_df_even['powerPlayPoints'] - all_df_even['powerPlayGoals']
all_df_even['shorthandedAssists'] = all_df_even['shPoints'] - all_df_even['shorthandedGoals']
all_df_even['evenAssists'] = all_df_even['assists'] - all_df_even['powerPlayAssists'] - all_df_even['shorthandedAssists']
all_df_even['evenPoints'] = all_df_even['assists'] + all_df_even['goals']
all_df_even['evenToi'] = all_df_even['toi'] - all_df_even['powerPlayToi'] - all_df_even['shorthandedToi']

all_df_even

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,gameId,gamesPlayed,fantasyPoints,specialTeams,evenGoals,powerPlayAssists,shorthandedAssists,evenAssists,evenPoints,evenToi
0,8478178,43,D. Raddysh,D,0,0,0,1,0,1,...,2023020001,1,0.9,0,0,0,0,0,0,1060
1,8475177,44,C. de Haan,D,0,0,0,0,0,2,...,2023020001,1,0.7,0,0,0,0,0,0,649
2,8480246,48,N. Perbix,D,0,0,0,-1,2,0,...,2023020001,1,0.6,0,0,0,0,0,0,702
3,8475167,77,V. Hedman,D,0,1,1,-1,0,0,...,2023020001,1,1.9,0,0,0,0,1,1,981
4,8478416,81,E. Cernak,D,0,0,0,0,0,6,...,2023020001,1,1.3,0,0,0,0,0,0,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17377,8475842,39,S. Carrick,C,0,0,0,0,0,1,...,2023020479,1,1.2,0,0,0,0,0,0,500
17378,8477527,44,R. Johnston,L,0,0,0,0,2,1,...,2023020479,1,0.6,0,0,0,0,0,0,349
17379,8479368,49,M. Jones,L,0,0,0,0,0,0,...,2023020479,1,0.0,0,0,0,0,0,0,675
17380,8478366,77,F. Vatrano,R,0,0,0,0,2,1,...,2023020479,1,0.8,0,0,0,0,0,0,704


In [8]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [9]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)

opp_even = opposition_stats(all_df_even)

opp_centers

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,169704,65,38,331,111,95,247.7,31,29,7.99,15,Anaheim Ducks
1,ARI,157065,43,29,324,170,82,200.9,30,16,6.70,30,Arizona Coyotes
2,BOS,167321,45,33,338,199,99,222.2,29,15,7.66,23,Boston Bruins
3,BUF,162158,60,39,324,189,81,240.3,32,18,7.51,25,Buffalo Sabres
4,CAR,164591,72,38,270,119,163,280.9,31,19,9.06,5,Carolina Hurricanes
5,CBJ,172774,70,52,376,115,107,285.1,32,14,8.91,6,Columbus Blue Jackets
6,CGY,175400,56,40,348,145,98,246.3,32,21,7.70,22,Calgary Flames
7,CHI,163940,64,52,352,168,75,273.0,30,26,9.10,4,Chicago Blackhawks
8,COL,169476,53,44,324,123,105,251.7,31,23,8.12,14,Colorado Avalanche
9,DAL,174417,61,45,345,181,113,265.6,30,10,8.85,7,Dallas Stars


In [10]:
# g_last_ten_df_team = pd.DataFrame()

# for index, row in team_names_only.iterrows():
#     temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
#     five_dates = temp_all['gameDate'].unique()[:10]
#     temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
#     g_last_ten_df_team = pd.concat([g_last_ten_df_team, temp_filtered])
    
# g_last_ten_df_team

# gopponent_stats_10 = g_last_ten_df_team.groupby('opponent').agg({
#     'toi': 'sum',
#     'goalsAgainst': 'sum',
#     'shots': 'sum',
#     'fantasyPoints': 'sum',
#     'gameDate': 'nunique',
#     'evSaves': 'sum',
#     'evShots': 'sum',
#     'shutout': 'sum',
#     'ppSaves': 'sum',
#     'ppShots': 'sum'
# })

# gopponent_stats_10['FPAPG'] = round(gopponent_stats_10['fantasyPoints'] / gopponent_stats_10['gameDate'], 2)
# gopponent_stats_10['FPAPG_Rank'] = gopponent_stats_10['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats_10 = gopponent_stats_10.rename_axis('abbreviation')

# gopponent_stats_10 = pd.merge(gopponent_stats_10, team_names_only, on='abbreviation')

# gopponent_stats_10

In [11]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,111419,79,903,89.8,31,665,719,1,123,145,2.90,7,Anaheim Ducks
1,ARI,107706,88,798,32.0,30,572,633,1,124,148,1.07,24,Arizona Coyotes
2,BOS,104015,88,900,30.4,29,652,718,0,129,151,1.05,25,Boston Bruins
3,BUF,114882,83,957,91.8,32,729,799,2,129,141,2.87,8,Buffalo Sabres
4,CAR,112307,96,1047,65.2,31,795,864,1,126,148,2.10,15,Carolina Hurricanes
5,CBJ,114800,92,941,81.8,32,698,776,2,130,144,2.56,11,Columbus Blue Jackets
6,CGY,115829,89,992,84.6,32,767,838,1,113,124,2.64,10,Calgary Flames
7,CHI,107343,67,815,107.6,30,616,672,2,101,112,3.59,2,Chicago Blackhawks
8,COL,111315,103,985,28.4,31,699,773,3,162,186,0.92,27,Colorado Avalanche
9,DAL,108216,100,878,14.6,30,648,722,2,103,124,0.49,30,Dallas Stars


In [12]:
player_bios.loc[player_bios['name'] == 'A. Ovechkin']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
177,8471214,A. Ovechkin,WSH,L,33107,28,44.6,4.85,1.59,0,54,85.86,Alex Ovechkin,1985-09-17,R,75,238,98.818864,LW,3101


In [13]:
team_names_only

last_five_df_team = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df_team = pd.concat([last_five_df_team, temp_filtered])
    
last_five_df_team

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,team,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams
17111,8476539,81,J. Marchessault,R,1,0,1,0,0,2,...,VGK,OTT,W,F,2023-12-17 00:00:00,Sunday 08:00 PM,2023020478,1,2.6,0
17102,8476905,20,C. Stephenson,C,1,0,1,1,0,0,...,VGK,OTT,C,F,2023-12-17 00:00:00,Sunday 08:00 PM,2023020478,1,2.6,1
17094,8480727,2,Z. Whitecloud,D,0,0,0,0,0,2,...,VGK,OTT,D,D,2023-12-17 00:00:00,Sunday 08:00 PM,2023020478,1,0.8,0
17095,8475188,3,B. McNabb,D,0,1,1,0,0,3,...,VGK,OTT,D,D,2023-12-17 00:00:00,Sunday 08:00 PM,2023020478,1,1.8,0
17096,8474565,7,A. Pietrangelo,D,0,0,0,2,0,1,...,VGK,OTT,D,D,2023-12-17 00:00:00,Sunday 08:00 PM,2023020478,1,1.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14081,8482117,27,L. Reichel,L,0,0,0,2,0,0,...,CHI,STL,W,F,2023-12-09 00:00:00,Saturday 08:00 PM,2023020416,1,0.0,0
14082,8481147,52,R. Johnson,C,0,0,0,1,5,2,...,CHI,STL,C,F,2023-12-09 00:00:00,Saturday 08:00 PM,2023020416,1,0.9,0
14084,8480252,70,C. Guttman,C,0,1,1,1,0,0,...,CHI,STL,C,F,2023-12-09 00:00:00,Saturday 08:00 PM,2023020416,1,1.4,0
14085,8474870,90,T. Johnson,C,0,0,0,-1,0,1,...,CHI,STL,C,F,2023-12-09 00:00:00,Saturday 08:00 PM,2023020416,1,1.5,0


In [14]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [15]:


top_players_by_team = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team

for index, row in top_players_by_team.iterrows():
    top_players_by_team.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
54,8474641,A. Henrique,ANA,C,5342,5,6,1,3,10,...,8,786,1078,4,19.5,C,F,13.14,3.90,94.67
365,8478873,T. Terry,ANA,R,5483,5,3,3,2,14,...,4,556,16,1,13.4,W,F,8.80,2.68,48.64
26,8473986,A. Killorn,ANA,L,5434,5,2,3,2,16,...,2,462,632,2,10.8,W,F,7.15,2.16,97.39
577,8481605,J. LaCombe,ANA,D,6341,5,0,1,0,3,...,18,161,1108,-2,10.6,D,D,6.02,2.12,99.88
109,8475764,C. Fowler,ANA,D,7689,5,0,3,1,6,...,10,746,1336,-1,9.1,D,D,4.26,1.82,52.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,8476880,T. Wilson,WSH,R,5851,5,2,2,1,14,...,7,926,641,1,12.7,W,F,7.81,2.54,46.85
329,8478440,D. Strome,WSH,C,5803,5,2,3,2,17,...,1,993,16,1,10.2,C,F,6.33,2.04,47.71
499,8480796,M. Fehervary,WSH,D,5764,5,1,1,0,3,...,11,10,672,2,9.7,D,D,6.06,1.94,96.83
92,8475343,N. Dowd,WSH,C,4668,5,2,0,0,6,...,5,8,929,2,7.4,C,F,5.71,1.48,99.62


In [16]:

worst_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nsmallest(10, 'fantasyPoints'))
worst_five.reset_index(level='team', drop=True, inplace=True)

for index, row in worst_five.iterrows():
    worst_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
worst_five = worst_five.loc[worst_five['available'] <= 40]    


for index, row in worst_five.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['gamesPlayed']} GP) "
)

<p><b>Frank Vatrano, R, Anaheim Ducks </b>(9.7% available): 4.4 fantasy points (5 GP) 
<p><b>Sean Durzi, D, Arizona Coyotes </b>(27.7% available): 0.2 fantasy points (1 GP) 
<p><b>Charlie McAvoy, D, Boston Bruins </b>(2.5% available): 0.5 fantasy points (1 GP) 
<p><b>Tage Thompson, C, Buffalo Sabres </b>(3.7% available): 3.5 fantasy points (5 GP) 
<p><b>Jeff Skinner, L, Buffalo Sabres </b>(27.2% available): 4.4 fantasy points (3 GP) 
<p><b>Brent Burns, D, Carolina Hurricanes </b>(23.3% available): 4.3 fantasy points (5 GP) 
<p><b>Boone Jenner, C, Columbus Blue Jackets </b>(31.5% available): 2.8 fantasy points (2 GP) 
<p><b>Johnny Gaudreau, L, Columbus Blue Jackets </b>(34.3% available): 5.7 fantasy points (5 GP) 
<p><b>Bowen Byram, D, Colorado Avalanche </b>(28.5% available): 5.0 fantasy points (5 GP) 
<p><b>Devon Toews, D, Colorado Avalanche </b>(23.4% available): 6.7 fantasy points (5 GP) 
<p><b>Jamie Benn, L, Dallas Stars </b>(32.0% available): 4.3 fantasy points (5 GP) 
<p><b>Dylan

In [25]:
best_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
best_five.reset_index(level='team', drop=True, inplace=True)

for index, row in best_five.iterrows():
    best_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
best_five = best_five.loc[(best_five['available'] >= 60) & (best_five['FPPG'] >= 1.7)]    


for index, row in best_five.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['gamesPlayed']} GP) {row['FPPG']} FPPG"
)

<p><b>Adam Henrique, C, Anaheim Ducks </b>(94.7% available): 19.5 fantasy points (5 GP) 3.9 FPPG
<p><b>Alex Killorn, L, Anaheim Ducks </b>(97.4% available): 10.8 fantasy points (5 GP) 2.16 FPPG
<p><b>Jackson LaCombe, D, Anaheim Ducks </b>(99.9% available): 10.6 fantasy points (5 GP) 2.12 FPPG
<p><b>Morgan Geekie, C, Boston Bruins </b>(98.6% available): 9.7 fantasy points (5 GP) 1.94 FPPG
<p><b>Kevin Shattenkirk, D, Boston Bruins </b>(92.3% available): 9.3 fantasy points (5 GP) 1.86 FPPG
<p><b>James van Riemsdyk, L, Boston Bruins </b>(91.5% available): 8.6 fantasy points (5 GP) 1.72 FPPG
<p><b>Zach Benson, L, Buffalo Sabres </b>(98.1% available): 9.4 fantasy points (5 GP) 1.88 FPPG
<p><b>Kyle Okposo, R, Buffalo Sabres </b>(99.6% available): 9.1 fantasy points (5 GP) 1.82 FPPG
<p><b>Connor Clifton, D, Buffalo Sabres </b>(98.0% available): 9.0 fantasy points (5 GP) 1.8 FPPG
<p><b>Stefan Noesen, R, Carolina Hurricanes </b>(96.9% available): 8.9 fantasy points (5 GP) 1.78 FPPG
<p><b>Yegor C

In [18]:
team_names_only


last_five_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df = pd.concat([last_five_df, temp_filtered])
    
last_five_df

,playerId,sweaterNumber,name,position,pim,goalsAgainst,toi,team,opponent,gameDate,...,win,saves,shots,evSaves,evShots,ppSaves,ppShots,gamesPlayed,fantasyPoints,decisionType
1011,8480313,36,L. Thompson,G,0,2,3183,VGK,OTT,2023-12-17 00:00:00,...,4,20,22,14,15,4,5,1,4.0,W
1010,8478499,33,A. Hill,G,0,1,385,VGK,OTT,2023-12-17 00:00:00,...,0,2,3,2,2,0,1,1,-1.6,L
946,8480313,36,L. Thompson,G,0,4,3553,VGK,BUF,2023-12-15 00:00:00,...,0,22,26,20,24,1,1,1,-3.6,L
925,8480313,36,L. Thompson,G,0,4,3671,VGK,CGY,2023-12-12 00:00:00,...,4,27,31,20,24,4,4,1,1.4,W
844,8480238,30,J. Patera,G,0,4,3900,VGK,SJS,2023-12-10 00:00:00,...,4,34,38,29,32,5,6,1,2.8,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,8482821,40,A. Soderblom,G,0,7,2608,CHI,SEA,2023-12-14 00:00:00,...,0,17,24,14,20,3,4,1,-10.6,L
981,8475852,34,P. Mrazek,G,0,0,992,CHI,SEA,2023-12-14 00:00:00,...,0,5,5,5,5,0,0,1,1.0,L
965,8475852,34,P. Mrazek,G,0,4,3600,CHI,EDM,2023-12-12 00:00:00,...,0,25,29,22,25,3,4,1,-3.0,L
839,8482821,40,A. Soderblom,G,0,4,3512,CHI,WSH,2023-12-10 00:00:00,...,0,23,27,20,24,3,3,1,-3.4,L


In [19]:
summary_stats_last5G = last_five_df.groupby('playerId').agg({
    'name': 'first',
    'team': 'last',
    'position': 'first',
    'toi': 'sum',
    'gamesPlayed': 'sum',
    'saves': 'sum',
    'shots': 'sum',
    'start': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last5G.sort_values('fantasyPoints', ascending=False).head(25)
summary_stats_last5G

for index, row in summary_stats_last5G.iterrows():
    summary_stats_last5G.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

summary_stats_last5G['FPP60'] = round(summary_stats_last5G['fantasyPoints'] / summary_stats_last5G['toi'] *3600, 2)
summary_stats_last5G

,playerId,name,team,position,toi,gamesPlayed,saves,shots,start,shutout,fantasyPoints,available,FPP60
0,8470594,M. Fleury,MIN,G,3493,1,36,40,1,0,-0.8,86.89,-0.82
1,8471734,J. Quick,NYR,G,7152,2,54,56,2,0,14.8,55.30,7.45
2,8473503,J. Reimer,DET,G,5888,2,41,48,1,0,-5.8,95.03,-3.55
3,8473575,S. Varlamov,NYI,G,7137,2,57,64,2,0,1.4,75.47,0.71
4,8474593,J. Markstrom,CGY,G,3600,1,33,34,1,0,8.6,55.74,8.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,8481611,P. Kochetkov,CAR,G,11095,3,76,79,3,0,18.2,64.85,5.91
65,8481692,D. Wolf,CGY,G,3626,1,34,39,1,0,-2.2,98.34,-2.18
66,8482221,D. Levi,BUF,G,11100,3,83,89,3,0,13.6,79.86,4.41
67,8482821,A. Soderblom,CHI,G,6120,2,40,51,2,0,-14.0,99.33,-8.24


In [20]:
top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
54,8474641,A. Henrique,ANA,C,5342,5,6,1,3,10,...,8,786,1078,4,19.5,C,F,13.14,3.90,94.67
365,8478873,T. Terry,ANA,R,5483,5,3,3,2,14,...,4,556,16,1,13.4,W,F,8.80,2.68,48.64
26,8473986,A. Killorn,ANA,L,5434,5,2,3,2,16,...,2,462,632,2,10.8,W,F,7.15,2.16,97.39
577,8481605,J. LaCombe,ANA,D,6341,5,0,1,0,3,...,18,161,1108,-2,10.6,D,D,6.02,2.12,99.88
109,8475764,C. Fowler,ANA,D,7689,5,0,3,1,6,...,10,746,1336,-1,9.1,D,D,4.26,1.82,52.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,8476880,T. Wilson,WSH,R,5851,5,2,2,1,14,...,7,926,641,1,12.7,W,F,7.81,2.54,46.85
329,8478440,D. Strome,WSH,C,5803,5,2,3,2,17,...,1,993,16,1,10.2,C,F,6.33,2.04,47.71
499,8480796,M. Fehervary,WSH,D,5764,5,1,1,0,3,...,11,10,672,2,9.7,D,D,6.06,1.94,96.83
92,8475343,N. Dowd,WSH,C,4668,5,2,0,0,6,...,5,8,929,2,7.4,C,F,5.71,1.48,99.62


In [21]:
df = lookahead_sked.copy()

df.rename(columns={'homeTeam': 'focusTeam', 'gameDate': 'date'}, inplace=True)
df_temp = df[['gameID', 'date', 'awayTeam', 'focusTeam', 'gameTime', 'gameDT']]
df_temp.rename(columns={'focusTeam': 'opponent', 'date': 'date', 'awayTeam': 'focusTeam'}, inplace=True)
df.rename(columns={'awayTeam': 'opponent'}, inplace=True)
df['H/A'] = 'H'
df_temp['H/A'] = 'A'
df = pd.concat([df, df_temp])
df.drop(columns=['winningGoalie'], inplace=True)

sked_stats = df
sked_stats.reset_index(inplace=True)
sked_stats = sked_stats.sort_values('gameTime')

sked_stats = sked_stats[['gameID', 'date', 'gameTime', 'focusTeam', 'H/A', 'opponent']]
# sked_stats['opponent'] = sked_stats.groupby('focusTeam')['opponent'].sum()
sked_stats


for index, row in sked_stats.iterrows():
    d = opp_defense.loc[opp_defense['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     drank = D.loc[D.index == row['focus']]['FPAPG_Rank'][0]
    c = opp_centers.loc[opp_centers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     crank = C.loc[C.index == row['focus']]['FPAPG_Rank'][0]
    w = opp_wingers.loc[opp_wingers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     wrank = W.loc[W.index == row['focus']]['FPAPG_Rank'][0]
    g = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
    
    sked_stats.at[index, 'D_FPAPG'] = d
#     sked_stats.at[index, 'D_RANK'] = drank
    sked_stats.at[index, 'C_FPAPG'] = c
#     sked_stats.at[index, 'C_RANK'] = crank
    sked_stats.at[index, 'W_FPAPG'] = w
#     sked_stats.at[index, 'W_RANK'] = wrank
    sked_stats.at[index, 'G_FPAPG'] = g
    
sked_summary = sked_stats.groupby('focusTeam').sum().sort_values('D_FPAPG', ascending=False)
sked_summary['D_rank'] = sked_summary['D_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['C_rank'] = sked_summary['C_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['W_rank'] = sked_summary['W_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['G_rank'] = sked_summary['G_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary

sked_summary.sort_values('D_rank', ascending=True, inplace=True)
sked_summary

d_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['position'] == 'D')]
    temp = temp.loc[temp['available'] >= 25]
    temp['D_rank'] = row['D_rank'].astype(int)
    d_adds = pd.concat([d_adds, temp])
    if row['D_rank'] == 5:
        break

sked_summary.sort_values('C_rank', ascending=True, inplace=True)
c_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'C')]
    temp = temp.loc[temp['available'] >= 25]
    temp['C_rank'] = row['C_rank'].astype(int)
    c_adds = pd.concat([c_adds, temp])
    if row['C_rank'] == 5:
        break
        
sked_summary.sort_values('W_rank', ascending=True, inplace=True)
w_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'W')]
    temp = temp.loc[temp['available'] >= 25]
    temp['W_rank'] = row['W_rank'].astype(int)
    w_adds = pd.concat([w_adds, temp])
    if row['W_rank'] == 5:
        break
        
sked_summary.sort_values('G_rank', ascending=True, inplace=True)
g_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = summary_stats_last5G.loc[(summary_stats_last5G['team'] == index)]
    temp = temp.loc[temp['available'] >= 25]
    temp['G_rank'] = row['G_rank'].astype(int)
    g_adds = pd.concat([g_adds, temp])
    if row['G_rank'] == 5:
        break
        
adds = pd.concat([d_adds, c_adds, w_adds])
adds['merged_rank'] = adds['D_rank'].combine_first(adds['C_rank']).combine_first(adds['W_rank'])
adds = adds.drop(['D_rank', 'C_rank', 'W_rank'], axis=1)
sked_stats

for index, row in adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPPG']} FPPG) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)
    
for index, row in g_adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPP60']} FPP60) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)

<p><b>David Savard, D, Montréal Canadiens </b>(66.9% available): 9.4 fantasy points (2.35 FPPG) CHIMIN
<p><b>Justin Barron, D, Montréal Canadiens </b>(98.2% available): 9.1 fantasy points (1.82 FPPG) CHIMIN
<p><b>Cam York, D, Philadelphia Flyers </b>(91.3% available): 11.1 fantasy points (2.22 FPPG) DETNSH
<p><b>Travis Sanheim, D, Philadelphia Flyers </b>(27.8% available): 10.2 fantasy points (2.55 FPPG) DETNSH
<p><b>Martin Fehervary, D, Washington Capitals </b>(96.8% available): 9.7 fantasy points (1.94 FPPG) CBJNYI
<p><b>Brenden Dillon, D, Winnipeg Jets </b>(92.0% available): 9.4 fantasy points (1.88 FPPG) BOSDET
<p><b>Dylan Strome, C, Washington Capitals </b>(47.7% available): 10.2 fantasy points (2.04 FPPG) CBJNYI
<p><b>Nic Dowd, C, Washington Capitals </b>(99.6% available): 7.4 fantasy points (1.48 FPPG) CBJNYI
<p><b>Sean Monahan, C, Montréal Canadiens </b>(83.0% available): 9.0 fantasy points (1.8 FPPG) CHIMIN
<p><b>Sean Couturier, C, Philadelphia Flyers </b>(29.8% available): 10

In [22]:
sked_summary

,gameID,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG,D_rank,C_rank,W_rank,G_rank
focusTeam,,,,,,,,,
MTL,4046041019,18.59,16.89,17.33,5.81,1,2,1,1
WSH,4046040995,18.12,17.71,16.88,4.18,4,1,2,2
DET,4046041005,16.30,14.04,14.73,3.99,6,6,6,3
ARI,2023020509,10.06,8.71,9.30,3.94,8,14,7,4
LAK,2023020497,9.90,8.65,7.52,3.36,11,15,24,5
CAR,2023020503,8.93,7.37,8.61,3.20,19,28,10,6
STL,2023020500,9.35,8.35,7.46,3.04,16,16,25,7
CBJ,2023020499,8.20,7.84,7.22,3.02,28,20,28,8
NYI,2023020496,8.20,7.84,7.22,3.02,29,21,29,9


In [23]:
lookahead_sked = lookahead_sked.sort_values('gameTime')
lookahead_sked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
632,2023020510,2023-12-22,Friday 07:00 PM,PHI,DET,2023-12-23 00:00:00,0
95,2023020511,2023-12-22,Friday 07:30 PM,EDM,NYR,2023-12-23 00:30:00,0
263,2023020512,2023-12-22,Friday 08:00 PM,BOS,WPG,2023-12-23 01:00:00,0
1286,2023020513,2023-12-22,Friday 08:30 PM,MTL,CHI,2023-12-23 01:30:00,0
1125,2023020499,2023-12-21,Thursday 07:00 PM,WSH,CBJ,2023-12-22 00:00:00,0
958,2023020498,2023-12-21,Thursday 07:00 PM,TOR,BUF,2023-12-22 00:00:00,0
382,2023020500,2023-12-21,Thursday 07:00 PM,STL,FLA,2023-12-22 00:00:00,0
425,2023020502,2023-12-21,Thursday 07:00 PM,NSH,PHI,2023-12-22 00:00:00,0
875,2023020503,2023-12-21,Thursday 07:00 PM,CAR,PIT,2023-12-22 00:00:00,0
712,2023020504,2023-12-21,Thursday 07:00 PM,VGK,TBL,2023-12-22 00:00:00,0


In [24]:
for index, row in lookahead_sked.iterrows():
    
    date_object_utc = datetime.strptime(row['gameDT'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    date_object_et = date_object_utc.astimezone(pytz.timezone("US/Eastern"))
    formatted_time = date_object_et.strftime("%I:%M p.m.").lstrip('0').replace("PM", "p.m.")
    
    date_string_et = date_object_et.strftime("%Y-%m-%d %H:%M:%S%z")
    date_object = datetime.strptime(date_string_et, "%Y-%m-%d %H:%M:%S%z")    
    day_of_week = date_object.strftime("%A")
    
    home_abbr = row['homeTeam']
    home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
    home_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == home_abbr]

    away_abbr = row['awayTeam']
    away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
    away_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == away_abbr]
    
    home_team = team_names_only.loc[team_names_only['abbreviation'] == home_abbr]['name'].iloc[0]
    away_team = team_names_only.loc[team_names_only['abbreviation'] == away_abbr]['name'].iloc[0]
        
    print(f"\033[1m{away_team} at {home_team}\033[0m")
    print(f"{formatted_time}")
    print()
    print(f"\033[1mForwards against {home_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {home_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {away_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in away_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {away_team} </b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print()
    print(f"\033[1mGoalies against {home_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
    for index, row in away_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 1)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print(f"\033[1mForwards against {away_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {away_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {home_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in home_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {home_team}</b>({round(row['available'], 1)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")

    print()
    print(f"\033[1mGoalies against {away_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
    for index, row in home_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 1)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print('-'*25)
    print()

Philadelphia Flyers at Detroit Red Wings
7:00 p.m.

Forwards against DET score: 15.3 FPAPG, Rank: 22
Defence against DET score: 8.98 FPAPG, Rank: 17
Top PHI Players Fantasy Points Last 5 
<p><b>Joel Farabee, L, Philadelphia Flyers </b>(86.8% available): 14.8 fantasy points (2.96 FPPG)
<p><b>Travis Konecny, R, Philadelphia Flyers </b>(3.5% available): 13.4 fantasy points (2.68 FPPG)
<p><b>Cam York, D, Philadelphia Flyers </b>(91.3% available): 11.1 fantasy points (2.22 FPPG)
<p><b>Travis Sanheim, D, Philadelphia Flyers </b>(27.8% available): 10.2 fantasy points (2.55 FPPG)
<p><b>Sean Couturier, C, Philadelphia Flyers </b>(29.8% available): 10.0 fantasy points (2.0 FPPG)

Goalies against DET score: 1.02 FPAPG, Rank: 26
Carter Hart (24.3% available) last five: 2 starts, FPP60: 7.1, FP: 14.2
Samuel Ersson (98.4% available) last five: 3 starts, FPP60: 5.12, FP: 15.8

Forwards against PHI score: 14.62 FPAPG, Rank: 26
Defence against PHI score: 7.56 FPAPG, Rank: 32
Top DET Players Fantasy Poi

In [25]:
temp_team_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:6]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    temp_team_df = pd.concat([temp_team_df, temp_filtered])
    
temp_team_df = temp_team_df.loc[(temp_team_df['team'] == 'STL') & (temp_team_df['position'] == 'D')]
temp_team_df = summary_statistics(temp_team_df)
# temp_team_df['powerPlayToi'] = f"{(temp_team_df['powerPlayToi'] // 60)}:{(temp_team_df['powerPlayToi'] % 60)}"
temp_team_df

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
4,8476892,C. Parayko,STL,D,8445,6,1,1,0,14,8,18,49,1270,1,14.2,D,D,6.05,2.37
1,8475181,N. Leddy,STL,D,8120,6,0,3,1,5,1,12,50,1421,1,10.1,D,D,4.48,1.68
3,8476792,T. Krug,STL,D,7898,6,0,5,0,16,3,6,1081,502,-1,9.9,D,D,4.51,1.65
2,8475753,J. Faulk,STL,D,8022,6,0,3,0,25,5,7,389,1072,2,9.5,D,D,4.26,1.58
0,8474618,M. Scandella,STL,D,3432,5,1,0,1,5,8,1,0,560,-1,4.3,D,D,4.51,0.86
5,8481006,T. Tucker,STL,D,1886,3,0,0,0,1,1,2,0,0,-2,1.2,D,D,2.29,0.40
6,8481059,S. Perunovich,STL,D,3409,4,0,0,0,5,1,2,564,9,-7,1.6,D,D,1.69,0.40


In [74]:
temp_player = all_df.loc[all_df['name'] == 'A. Svechnikov']
temp_player = temp_player.sort_values('gameDate', ascending=True).head(10)
summary_statistics(temp_player)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8480830,A. Svechnikov,CAR,R,9928,10,0,6,1,21,23,3,1085,4,0,12.4,W,F,4.5,1.24


In [29]:
player_bios.sort_values('fantasyPoints', ascending=False, inplace=True)
player_bios['rank'] = player_bios['fantasyPoints'].rank(ascending=False, method='first')
player_bios.loc[player_bios['team'] == 'DAL']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,rank
47,8470794,J. Pavelski,DAL,C,28282,29,66.8,8.50,2.30,0,...,121.90,Joe Pavelski,1984-07-11,R,73,196,90.676521,RW,3451,44.0
81,8480036,M. Heiskanen,DAL,D,44015,29,60.0,4.91,2.07,0,...,109.71,Miro Heiskanen,1999-07-18,L,74,197,96.627153,D,4233536,82.0
110,8480027,J. Robertson,DAL,L,31402,29,56.1,6.43,1.93,0,...,102.29,Jason Robertson,1999-07-22,L,75,202,99.646106,LW,4233875,99.0
98,8478449,R. Hintz,DAL,C,27441,27,53.3,6.99,1.97,0,...,104.41,Roope Hintz,1996-11-17,L,75,212,95.648096,C,3904183,123.0
120,8475168,M. Duchene,DAL,C,28386,28,52.1,6.61,1.86,0,...,98.58,Matt Duchene,1991-01-16,L,73,210,40.444855,C,5161,131.0
111,8481581,T. Harley,DAL,D,28566,26,50.0,6.30,1.92,0,...,101.76,Thomas Harley,2001-08-19,L,76,206,8.927455,D,4565239,149.0
171,8476902,E. Lindell,DAL,D,34615,29,47.6,4.95,1.64,0,...,86.92,Esa Lindell,1994-05-23,L,75,216,35.186764,D,3069352,177.0
185,8482740,W. Johnston,DAL,C,29368,29,47.1,5.77,1.62,0,...,85.86,Wyatt Johnston,2003-05-14,R,74,184,31.114269,C,4874740,181.0
210,8473994,J. Benn,DAL,L,26273,29,45.5,6.23,1.57,0,...,83.21,Jamie Benn,1989-07-18,L,74,205,68.335702,LW,3998,198.0
234,8478975,M. Marchment,DAL,L,25110,29,43.8,6.28,1.51,0,...,80.03,Mason Marchment,1995-06-18,L,77,216,9.461342,LW,4272192,219.0


In [76]:
last_five_df_team
hurricanes = last_five_df_team.loc[last_five_df_team['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
12,8478427,S. Aho,CAR,C,5606,5,3,3,1,21,2,1,763,467,4,12.3,C,F,7.90,2.46
10,8477962,B. Lemieux,CAR,L,573,1,1,0,0,3,0,0,0,0,1,2.3,W,F,14.45,2.30
16,8480830,A. Svechnikov,CAR,R,3694,4,1,4,3,12,5,0,578,40,1,9.2,W,F,8.97,2.30
6,8476882,T. Teravainen,CAR,L,5076,5,1,3,2,13,0,3,700,347,2,8.8,W,F,6.24,1.76
11,8478047,M. Bunting,CAR,L,4179,5,2,3,1,8,0,0,396,0,5,8.3,W,F,7.15,1.66
5,8476869,B. Skjei,CAR,D,6415,5,2,0,1,10,1,5,27,730,0,8.1,D,D,4.55,1.62
0,8470613,B. Burns,CAR,D,6283,5,0,3,2,14,2,4,713,527,2,7.6,D,D,4.35,1.52
4,8476474,S. Noesen,CAR,R,3596,5,2,1,0,10,6,1,410,3,4,7.1,W,F,7.11,1.42
14,8480039,M. Necas,CAR,C,4852,5,0,4,0,12,3,3,522,4,1,7.0,C,F,5.19,1.40
18,8482093,S. Jarvis,CAR,C,5583,5,1,3,0,5,3,2,745,728,5,6.8,C,F,4.38,1.36


In [77]:
top_players_by_team.sort_values('fantasyPoints', ascending=False).head(60)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
319,8478402,C. McDavid,EDM,C,6123,5,2,12,6,10,...,5,888,45,1,22.9,C,F,13.46,4.58,0.12
146,8476453,N. Kucherov,TBL,R,6538,5,2,9,6,28,...,1,1539,0,-1,19.3,W,F,10.63,3.86,0.13
428,8479675,T. Moore,LAK,L,5159,5,5,2,2,17,...,5,576,421,5,17.7,W,F,12.35,3.54,45.64
511,8480865,N. Dobson,NYI,D,8271,5,0,6,2,9,...,18,817,449,3,17.4,D,D,7.57,3.48,4.40
357,8478550,A. Panarin,NYR,L,5797,5,5,4,3,17,...,0,757,4,1,17.2,W,F,10.68,3.44,1.88
560,8481559,J. Hughes,NJD,C,6946,5,3,5,2,38,...,2,1169,17,3,17.0,C,F,8.81,3.40,0.29
238,8477492,N. MacKinnon,COL,C,7120,5,2,5,3,19,...,8,1451,7,2,16.7,C,F,8.44,3.34,0.10
592,8482089,J. Neighbours,STL,L,5519,5,5,0,1,17,...,5,897,282,-2,16.1,W,F,10.50,3.22,91.74
114,8475786,Z. Hyman,EDM,L,4605,4,4,4,2,23,...,1,643,12,4,16.1,W,F,12.59,4.03,7.72
265,8477934,L. Draisaitl,EDM,C,6048,5,4,3,5,12,...,2,898,274,-1,16.0,C,F,9.52,3.20,0.10


In [78]:
preNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] <= '2023-11-12')]
postNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] > '2023-11-12')]

preNichSum = summary_statistics(preNich)
postNichSum = summary_statistics(postNich)

postNichSum

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8474884,M. Hoffman,SJS,C,9174,10,6,2,2,17,4,5,937,0,1,19.6,C,F,7.69,1.96


In [79]:
summary_statistics(all_df).sort_values('FPP60', ascending=False).head(60)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
637,8481559,J. Hughes,NJD,C,21649,17,9,21,14,82,4,6,3676,64,0,57.6,C,F,9.58,3.39
156,8476453,N. Kucherov,TBL,R,31088,24,15,25,20,117,13,9,5895,10,-3,82.5,W,F,9.55,3.44
235,8477380,J. Brodzinski,NYR,C,2451,4,0,4,0,9,0,3,184,0,2,6.4,C,F,9.40,1.60
309,8477956,D. Pastrnak,BOS,R,28095,24,14,22,15,114,24,4,5553,119,8,73.3,W,F,9.39,3.05
4,8470794,J. Pavelski,DAL,C,21316,22,11,13,7,54,20,19,3439,71,1,55.4,C,F,9.36,2.52
167,8476468,J. Miller,VAN,C,30022,25,13,23,17,56,52,19,6256,2712,11,77.8,C,F,9.33,3.11
594,8480990,D. Chisholm,WPG,D,778,1,0,1,0,0,0,2,113,0,0,2.0,D,D,9.25,2.00
395,8478550,A. Panarin,NYR,L,26852,23,15,20,14,89,1,3,4426,4,3,67.5,W,F,9.05,2.93
420,8479318,A. Matthews,TOR,C,28457,22,16,9,8,93,31,28,4624,1593,0,71.4,C,F,9.03,3.25
306,8477952,R. Fabbri,DET,C,8852,11,7,2,3,18,4,5,1019,52,-1,22.2,C,F,9.03,2.02
